In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm; tqdm.pandas()
pd.options.display.max_columns = 202
pd.options.display.max_rows = 300

In [2]:
train = pd.read_pickle('data/train_original.pkl')
test = pd.read_pickle('data/test_original.pkl')
special_cols = [col for col in train.columns if train[col].dtype != np.float64]
feature_cols = [col for col in train.columns if col not in special_cols]

In [3]:
all_df = pd.concat([train[feature_cols], test[feature_cols]], axis=0).reset_index(drop=True)

In [4]:
cumulative_vc = all_df[feature_cols[0]].value_counts()
all_df[feature_cols[0]+'_cum_vc'] = all_df[feature_cols[0]].map(cumulative_vc)

for col in tqdm(feature_cols[1:]):
    vc = all_df[col].value_counts()
    comm_idx = np.intersect1d(vc.index, cumulative_vc.index)
    cumulative_vc.loc[comm_idx] += vc.loc[comm_idx]
    cumulative_vc.append(vc.loc[vc.index[~vc.index.isin(comm_idx)]])
    all_df[feature_cols[0]+'_cum_vc'] = all_df[col].map(cumulative_vc)

100%|████████████████████████████████████████████████████████████████████████████| 199/199 [00:34<00:00,  5.53it/s]


In [5]:
all_df.iloc[:train.shape[0]].to_pickle('features/cumulative_vc_train.pkl')
all_df.iloc[train.shape[0]:].reset_index(drop=True).to_pickle('features/cumulative_vc_test.pkl')